In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import pickle
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import pandas as pd
import numpy as np
import os

In [ ]:
path = 'dataset/'

In [ ]:
dataframe = pd.read_csv(path+'train.csv') 

In [ ]:
dataframe.sample()

In [ ]:
images = os.listdir(path+'train/')

In [ ]:
class data_Label_Combiner:

    def Data_combiner(self,images, labels, path):
        x_train=[]
        y_train=[]
        for each in images:
            x_train.append(np.asarray(Image.open(path+'train/'+each).resize((32,32)), dtype='int32'))
#             y_train.append(np.asarray(binarizer.transform(labels[labels['ID']==each]['Class']),dtype='float32'))
            y_train.append(np.asarray(labels[labels['filename']==each]['label']))
        return x_train, y_train

In [ ]:
dlc= data_Label_Combiner()
x_train,y_train = dlc.Data_combiner(images,dataframe,path)

In [ ]:
x_train[0].shape

In [ ]:
y_train = np.array(y_train)
x_train = np.array(x_train)
print(x_train.shape)
# x_train = x_train.reshape(-1,32,32,3)

In [ ]:
# le = LabelEncoder()
# le.fit(y_train)
# # y_train = le.transform(y_train)

In [ ]:
y_train = np_utils.to_categorical(y_train)

In [ ]:
y_train.shape

In [ ]:
x_train, x_cross_val, y_train, y_cross_val = train_test_split(x_train,y_train, test_size = 0.1)

In [ ]:
x_train.shape

In [ ]:
# train_generator = ImageDataGenerator(rotation_range=0.2,zoom_range=0.3, shear_range=0.3, vertical_flip=True, rescale=1.0/255)

train_generator = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    zoom_range=0.3,
    shear_range=0.3,
    rescale=1.0/255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

val_generator = ImageDataGenerator(rescale=1.0/255)

In [ ]:
train_generator.fit(x_train)
val_generator.fit(x_cross_val)

In [ ]:
classifier = Sequential()

In [ ]:
classifier.add(Convolution2D(32,3,3, input_shape = (32,32,4),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
classifier.add(Flatten())

In [ ]:
classifier.add(Dense(output_dim=120, activation='relu'))
classifier.add(Dense(output_dim =10 ,activation='softmax'))

In [ ]:
classifier.compile(optimizer='adam',loss = 'categorical_crossentropy' ,metrics=['accuracy'])

In [ ]:
# fits the model on batches with real-time data augmentation:
classifier.fit_generator(train_generator.flow(x_train,y_train),
                    steps_per_epoch=len(x_train),
                         epochs=10, validation_data=val_generator.flow(x_cross_val,y_cross_val), validation_steps=len(x_cross_val)
                    )

In [ ]:
img = Image.fromarray(x_train[7], 'RGB').resize((256,256))
img.show()

In [ ]:
y_train[7]

In [ ]:
y_train.shape

In [ ]:
with open('pickle/model.pkl','wb') as f:
#     pickle.dump(le,f)
    pickle.dump(val_generator,f)
    pickle.dump(classifier,f)